In [152]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from ast import literal_eval
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import os
import sympy as sym
from sympy.parsing import mathematica

In [153]:
RESULTS_DIR = "final_results"
benchmarks = [
    "rational/plain",
    "sqrt/plain",
    "rational_sqrt/plain",
]

In [154]:
def process_output(row):
    if pd.isnull(row):
        return (None, None)

    try:
        row = float(row)
        return (row, row)
    except:
        pass

    try:
        return literal_eval(str(row).strip())
    except Exception as e:
        pass
    
    try:
        for w in ['integrate', 'boole']:
            if w in row.lower():
                return (None, None)        
        
        row = row.replace("[",'(').replace("]", ")").replace("L", "l").replace("E(p_) =", "").replace("^", "**").strip()
        row = sym.parse_expr(row).evalf()
        
        return (row,row)
        
    except Exception as e:
        print(e, row)
    
        
    return (None, None)

In [155]:
def process_time(row):
    if pd.isnull(row):
        return None

    try:
        row = float(row)
        return row
    except:
        pass

    try:
        return literal_eval(str(row).strip())[0]
    except Exception as e:
        pass
        
    except Exception as e:
        print(e, row)
    
        
    return None

In [156]:
def process_benchmark(benchmark):
    df_list = []
    for f_name in os.listdir(os.path.join(RESULTS_DIR, benchmark)):
        tool_name =  f_name.split("_")[-2]
        df_ = pd.read_csv(os.path.join(RESULTS_DIR, benchmark, f_name))
        df_['tool'] = tool_name
        df_list.append(df_[['index', 'tool','output', 'time']])
    df = pd.concat(df_list)

    df['output'] = df['output'].apply(process_output)
    df['time'] = df['time'].apply(process_time)
    
    df['lower'], df['upper'] = df['output'].apply(lambda o: o[0]),df['output'].apply(lambda o: o[1])
    df['error'] = df['upper']-df['lower']
    df['benchmark'] = benchmark
    df['is_solved'] = ~pd.isnull(df['error'])
    return df

In [157]:
df_b_list = []
for b in benchmarks:
    df_b_list.append(process_benchmark(b))

df_b = pd.concat(df_b_list)

In [158]:
df_b

,index,tool,output,time,lower,upper,error,benchmark,is_solved
0,0,faza,"(0.6209815507002485, 0.7190312692318667)",95.899110,0.620982,0.719031,0.09805,rational/plain,True
1,1,faza,"(1.3035252399983923, 1.4021111825192791)",2.971019,1.303525,1.402111,0.098586,rational/plain,True
2,2,faza,"(0, 0.016205584149650985)",11.460795,0,0.016206,0.016206,rational/plain,True
3,3,faza,"(0.47723272447183107, 0.5754865206866198)",16.182557,0.477233,0.575487,0.098254,rational/plain,True
4,4,faza,"(1.0119278375331635, 1.1080609820988139)",17.220680,1.011928,1.108061,0.096133,rational/plain,True
...,...,...,...,...,...,...,...,...,...
25,25,volesti,"(None, None)",0.000045,NaN,NaN,NaN,rational_sqrt/plain,False
26,26,volesti,"(None, None)",0.000045,NaN,NaN,NaN,rational_sqrt/plain,False
27,27,volesti,"(None, None)",0.000045,NaN,NaN,NaN,rational_sqrt/plain,False
28,28,volesti,"(None, None)",0.000052,NaN,NaN,NaN,rational_sqrt/plain,False


In [159]:
prop = 'error'
method = 'std'

df_ = df_b[[prop, 'tool', 'benchmark']].groupby(['benchmark','tool']).agg({prop:[method]}).reset_index()
df_.columns = df_.columns.droplevel(1)
df_ = df_.pivot_table(columns='tool', index='benchmark', values=prop)
df_

tool,faza,gubpi,latte,psi,volesti
benchmark,,,,,
rational/plain,0.015133,2.130043,0.0,0.0,0.295519
rational_sqrt/plain,0.040102,0.000814,NaN,NaN,NaN
sqrt/plain,0.000364,0.455869,NaN,0.0,NaN


In [160]:
df_.to_csv(os.path.join(RESULTS_DIR, f'{prop}_{method}.csv'))